<a href="https://colab.research.google.com/github/bhomaidan1990/Computer_Vision_Labs/blob/main/CV_Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision Course

## Lab (2)


In [24]:
##########################
# @Authors:              #
#     Belal Hmedan       #
# &   Junyi ZHONG        #
##########################

#========================
# Import Libraries
#========================
import os
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import cv2
import time
import math

### Download the Dataset and related files

In [25]:
#=======================
# Download the dataset
#=======================
!rm originalPics.tar.gz
!rm FDDB-folds.tgz
!rm test_db.zip
!wget http://tamaraberg.com/faceDataset/originalPics.tar.gz
!wget http://vis-www.cs.umass.edu/fddb/FDDB-folds.tgz
#-----------------------
# Test_Data
#==================
!wget http://crowley-coutaz.fr/jlc/Courses/2020/GVR.VO/test_db.zip
!mkdir test && unzip test_db.zip -d test
#-----------------------------------------------------------
# Extract the data
#==================
!rm -rf Dataset & rm -rf FDDB-folds & rm -rf sample_data & rm -rf NewDataset
t0 = time.time()
!mkdir Dataset && tar xvzf originalPics.tar.gz -C Dataset
!mkdir FDDB-folds && tar xvzf FDDB-folds.tgz -C FDDB-folds
t1 = time.time()
print('DeCompressing Time is : ',t1-t0," seconds")
# create the output dataset folder
!mkdir NewDataset

Streaming output truncated to the last 5000 lines.
2003/07/07/big/img_908.jpg
2003/07/01/big/img_250.jpg
2003/05/28/big/img_52.jpg
2002/11/06/big/img_476.jpg
2003/04/17/big/img_859.jpg
2002/09/30/big/img_903.jpg
2002/11/20/big/img_854.jpg
2003/02/23/big/img_874.jpg
2003/06/12/big/img_29.jpg
2003/03/10/big/img_518.jpg
2002/07/24/big/img_789.jpg
2003/03/10/big/img_327.jpg
2003/06/03/big/img_331.jpg
2002/09/04/big/img_13147.jpg
2002/07/23/big/img_306.jpg
2002/10/17/big/img_577.jpg
2002/08/08/big/img_981.jpg
2002/11/25/big/img_499.jpg
2002/12/18/big/img_347.jpg
2002/07/22/big/img_602.jpg
2003/07/17/big/img_341.jpg
2002/11/18/big/img_608.jpg
2003/07/02/big/img_367.jpg
2003/05/23/big/img_411.jpg
2002/12/10/big/img_717.jpg
2002/11/01/big/img_694.jpg
2002/11/26/big/img_258.jpg
2002/09/18/big/img_884.jpg
2002/10/22/big/img_1198.jpg
2003/03/10/big/img_748.jpg
2002/12/06/big/img_921.jpg
2003/07/18/big/img_30.jpg
2002/11/24/big/img_830.jpg
2002/11/01/big/img_193.jpg
2002/07/26/big/img_207.jpg
2003

### 1. Dataset Construction

**Building Indexed Data Structure for Accessing the Ground Truth easily.**

In [46]:
#========================
# Paths Initialization
#========================
# Define the Path of the original FDDB, and the output path

# datasetPath = "/media/desertghost/WD/2_Computer_vision/Lecture_1/Lab_1/Dataset/Datast/"
# outPath =     "/media/desertghost/WD/2_Computer_vision/Lecture_2/Lab_2/NewDataset/"

datasetPath = "Dataset"
outPath     = "NewDataset"

# Specify The required folds

# foldsPath = "/media/desertghost/WD/2_Computer_vision/Lecture_1/Lab_1/Dataset/FDDB-folds/"

foldsPath = "FDDB-folds/FDDB-folds/"
#=======================
foldsList  = os.listdir(foldsPath)
foldsList.sort()
folds = foldsList[1::2] # Evens, image names
annot = foldsList[0::2] # Odds, annotaions
# print(folds, '\n \n ', annot)
#------------------------
# missed backgrounds
miss = 0
#------------------------
# Get Annotation Function
#========================
def getAnnot(imgName, annotPath):
    '''
    Function to get the annotations in nd-array format for a specified image.
    
    Parameters:
    @Param: imgName : the specified image path(inside the DB).
    @Param: annotPath : the path to the txt file where the annotations stored.
    
    returns: nd-array with shape m x 5, : m num of the detected faces in the image.
             (5): max_radius | min_radius | angle | center_x | center_y
    '''
    # Initialization
    Annotaions = []
    AnnotData = []
    linIndex = -2
    numFaces = 0

    # Annot Search Loop
    AnnotFile = open(annotPath, "r")
    for idx, lin in enumerate(AnnotFile):
        if(re.findall(imgName,lin) != []):
            linIndex = idx
            # print(imgName, idx)
        if (idx == (linIndex + 1) ):
            numFaces = int(lin)
            # print("number of faces is: ", numFaces)
        if linIndex != -2:
            if idx in range(linIndex + 2, linIndex + 2 + numFaces):
                Annotaions.append(lin)
    # print(Annotaions)
    for Annotaion in Annotaions:
        AnnotData.append(list(filter(None, list(re.split("\s", Annotaion)))))
    AnnotData = np.array(AnnotData, dtype=float )
    AnnotData = AnnotData[...,:-1]
    # Close the txt file
    AnnotFile.close()

    return AnnotData
#----------------------------------------
# Build Data structure for easy indexing
# Dict[key = fold_num, value = dict[key = image name, value = nd-array]]
#========================================
DS = {}
for idx, fold in enumerate(folds):
    foldPath = os.path.join(foldsPath, fold)
    txtFile = open(foldPath, "r")
    imgDict = {}
    for lin in txtFile:
        imName = lin[:-1]
        # this to find only the string which ends with ths pattern
        imName = imName + '$'
        annotDir = os.path.join(foldsPath,annot[idx])
        # print("There",imName, annotDir)
        imgDict[lin[:-1]] = getAnnot(imName,annotDir)
        # print(imgList[0])
    # Close the txt file
    txtFile.close()
    DS["fold_{}".format(str(idx+1))] = imgDict

**function to Extract Faces and Backgrounds from the images**

In [49]:
#---------------------------------------------
# Extract faces, and Backgrounds from an image
#=============================================
def extData(imgPath, annots, foldPath, index):
    '''
    Function to Extract faces, and Backgrounds from an image.
    
    Parameters:
    @Param: imgPath : the specified image path(inside the DB).
    @Param: annots  : nd-array with shape m x 5, : m num of the detected faces in the image.
            (5): max_radius | min_radius | angle | center_x | center_y
    @Param: foldPath : the fold path where to save the extracted images.
    @Param: index : a number to start naming the faces, backgrounds from it.
    
    saves: m nd-array (m Faces), and m nd-array (m Background) in the foldPath
    '''
    fullImagePath = os.path.join(datasetPath, imgPath + '.jpg').replace('\\','/')
    img = cv2.imread(fullImagePath)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # create a mask with zeros in "faces" and bonudary
    mask = np.zeros(img.shape[:2], dtype=np.uint8)
    mask[1:-1, 1:-1] = 1
    #------------------
    # Loop the Faces
    #------------------
    for row in range(annots.shape[0]):
        # Initialize the Variables
        xc = math.floor(annots[row][3])
        yc = math.floor(annots[row][4])
        Rmax = annots[row][0] 
        Rmin =  annots[row][1]
        theta =  annots[row][2]
        # in case of rotation angles more than 45 degrees swap radius
        if(theta > 45):
            R = Rmax
            Rmax = Rmin
            Rmin = R
        # Rectangle Borders 
        st = math.sin(theta)
        st = st if st >= 0 else -st
        if(st == 0):
            st = 1
        # if(st >=(1/math.sqrt(2))):
        #     R = Rmax
        #     Rmax = Rmin
        #     Rmin = R
        # represents the top left corner of rectangle 
        x0 = math.floor(xc - Rmin * st)
        y0 = math.floor(yc - Rmax * st)
        x0 = x0 if x0 > 0 else 0
        y0 = y0 if y0 > 0 else 0
        # represents the bottom right corner of rectangle 
        x1 = math.floor(xc + Rmin * st)
        y1 = math.floor(yc + Rmax * st)
        x1 = x1 if x1 < img.shape[1] else img.shape[1]
        y1 = y1 if y1 < img.shape[0] else img.shape[0]
        # Crop the face in rectangular window
        # print("x0 = ", x0, " y0 = ", y0," x1 = ", x1," y1 = ", y1," sin(theta) = ", st)
        # print("xc = ", xc, " yc = ", yc," Rmax = ", Rmax," Rmin = ", Rmin," theta = ", theta)
        # print("image shape:", img.shape[:2])
        face = img[y0:y1, x0:x1,:]

        # make a directory to save the faces inside.
        foldfacedir = os.path.join(foldPath,"face")
        if(not os.path.isdir(foldfacedir)):
            os.mkdir(foldfacedir)
        facePath = os.path.join(foldfacedir, "face_{}.jpg".format(str(index+row)))
        # print(face.shape)
        cv2.imwrite(facePath, face)
        
        # Crop the face in rectangular window
        mask[y0:y1, x0:x1] = 0
        # ----- End of For Loop ----
    # once we have a map we can compute the distance of each non-face pixel to the nearest face
    dst = cv2.distanceTransform(mask, cv2.DIST_L1, 3)

    # pixels that are closer than 10 pixels (20//2) to a face, cannot be considered as good candidates.
    # If you allow for IoU > 0 this can be relaxed a little.
    dst[dst<10] = 0

    # linear indices of pixels
    idx = np.arange(np.prod(img.shape[:2]))

    # sample centers
    centers = []
    if(dst.sum()):
    	centers = np.random.choice(idx, size=annots.shape[0], replace=False, p=dst.flatten()/dst.sum())

    # create the background rectangles
    backgrounds = []
    for i, c in enumerate(centers):
        r = int(np.floor(dst.flat[c]))
        r = r if (r != 10) else 11
        # sample possible R from 10 to max possible
        r = np.random.choice(range(10,r))  
        [y, x] = np.unravel_index(c, img.shape[:2])
        backgrounds.append((y-r, x-r, y+r, x+r))
    # make a directory to save the backgrounds inside.
    backgroundfolddir = os.path.join(foldPath,"background")
    if(not os.path.isdir(backgroundfolddir)):
        os.mkdir(backgroundfolddir)
    #------------------
    # Loop the Backgrounds
    #------------------
    #------------------
    # Loop the Backgrounds
    #------------------
    global miss

    if(len(backgrounds)>= annots.shape[0]):

    	miss -= (len(backgrounds) - annots.shape[0])
    	
    	for row in range(len(backgrounds)):

            yb0 = backgrounds[row][0]
            xb0 = backgrounds[row][1]
            yb1 = backgrounds[row][2]
            xb1 = backgrounds[row][3]
            background = img[yb0:yb1, xb0:xb1,:]

            # print(miss, " backgrounds missed")

            # Save the background to the Dataset
            backgroundPath = os.path.join(backgroundfolddir, "background_{}.jpg".format(str(index+row)))
            cv2.imwrite(backgroundPath, background)
    
    elif((len(backgrounds)< annots.shape[0]) and len(backgrounds)):
    	
    	miss += annots.shape[0] - len(backgrounds)

    	for row in range(len(backgrounds)):
    		yb0 = backgrounds[row][0]
    		xb0 = backgrounds[row][1]
    		yb1 = backgrounds[row][2]
    		xb1 = backgrounds[row][3]
    		background = img[yb0:yb1, xb0:xb1,:]

    		# print(miss, " backgrounds missed")
    		
    		# Save the background to the Dataset
    		backgroundPath = os.path.join(backgroundfolddir, "background_{}.jpg".format(str(index+row)))
    		cv2.imwrite(backgroundPath, background)

    else:
    	miss += annots.shape[0]
    	# print(miss, " backgrounds missed")
#---------------------------------------------

**Extract Faces and Backgrounds fold by fold**

In [50]:
# Create Folds files in the New Dataset
index = 0
foldMiss = []
for fld in DS:
	miss = 0
	foldOutPath = os.path.join(outPath,fld)
	if(not os.path.isdir(foldOutPath)):
	    # print(outPath,'\n',foldOutPath)
	    os.mkdir(foldOutPath)
	for imag in DS[fld]:
		# print(imag)
		Anno = DS[fld][imag]
		extData(imag, Anno, foldOutPath, index)
		index += Anno.shape[0]
	print(fld,'miss is: ', miss)
	foldMiss.append(miss)

fold_1 miss is:  0
fold_2 miss is:  0
fold_3 miss is:  1
fold_4 miss is:  0
fold_5 miss is:  0
fold_6 miss is:  0
fold_7 miss is:  0
fold_8 miss is:  0
fold_9 miss is:  0
fold_10 miss is:  0


## 2. MultiLayer Perceptron (MLP) Network

Builing MLP using keras

In [ ]:
import keras
from keras import layers
from keras import Sequential

Pictures are in different size, img_window set up for all the same size, 8 by 8 is the minimun reconginizable face

In [ ]:
img_window = (8,8,1)
input_shape = np.prod(img_window)

2 dense neurons

In [ ]:
simpleMLP = keras.Sequential(
    [layers.Input(shape=img_window),
     layers.Flatten(),
     layers.Dense(input_shape//2,activation="relu"),
     layers.Dense(input_shape//2,activation="relu"),
     layers.Dense(2,activation="softmax"),
     ]
)
simpleMLP.summary()

In [ ]:
batch_size = 128
epochs = 5

simpleMLP.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])  #categorical

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

valid_datagen =ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    'classification_db/train_db',
    target_size=img_window[:2],
    batch_size=batch_size,
    class_mode=categorical,
    color_model='grayscale')

validation_generator = valid_datagen.flow_from_directory(
    'classification_db/valid_db',
    target_size=Img_window[:2],
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale')

In [ ]:
simpleMLP.fit(
    train_generator,
    steps_per_epoch=500,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=200)

In [ ]:
class_names = glob("classification_db/test_db/*) #Reads all the folders in which Images are present
class_names = [c.split('/')[-1] for c in class_names]

class_names = sorted(class_names) # Sorting them
name_id_map = dict(zip(range(len(class_names)), class_names))
name_id_map

In [ ]:
test_img_example = "classification_db/test_db/face/0010.jpg"
groundtruth_classname = test_img_example.split('/')[-2]

print('Original image:')

In [ ]:
img = keras.preprocessing.image_load_img(test_img_example,
                                         target_size = img_window,
                                         color_mode = 'grayscale')

print('loaded and resized Image:')
plt.figure(flgslze= (2,2))  
pit.imshow(img)

In [ ]:
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array,0)  #create batch axis
prediction = simpleMLP.predict(img_array)

predicted_class = predictions.argumax()
predicted_classname = name_id_map[pridicted_class]
print(test_img_example)
print('Predicted :',predicted_classname)
print('Groundtruth:',Groundtruth_classname)    

## Evaluuating the model

In [ ]:
predictions_dict = ()
for testfile in glob('classfication_db/test_db/*/*.jpg'):
  img = keras.preprocessing.image.load_img(testfile,target_size=img_window)
  img_array = keras.preprocessing.iamge.img_to_array(img)
  img_array = tf.expand_dims(img_array,0）  # create batch axis

  predictions = simpleMLP.predict(img_array)
  predictions_dict[testfile] = predictions[0]                           
                             
                            